In [ ]:
import ifcopenshell
import ifcopenshell.util.element
from collections import Counter

# GO TO MAIN SCRIPT, LINE 99 TO INSERT FILE NAME

# -----------------------------
# SETTINGS
# -----------------------------
CLAIMED_WINDOWS_DB = [
    {
        "name": "Window set: 2 x Udskifte fast vindue af træ, 1.188 x 1.188 mm, 3 lags; 1 x Udskifte vendevindue af træ, 1.188 x 1.188, 3 lags",
        "width_mm": 1188,
        "height_mm": 1188,
        "material": "Wood",
        "count": 330,
        "units_per_window": 3,  # 3 windows combined to create 1 claimed window
        "price_per_window": 18005.60  # Price per "assembled" window
    },
    # Add more claimed windows here if needed
]

# Window price database by area (type names simplified, price pr. m2 is based on average derived from molio database)
WINDOW_PRICE_DB = {
    "small": {"min_area": 0, "max_area": 1.5, "price_per_m2": 4000},
    "medium": {"min_area": 1.5, "max_area": 3.0, "price_per_m2": 6100},
    "large": {"min_area": 3.0, "max_area": float("inf"), "price_per_m2": 9000},
}

# -----------------------------
# FUNCTIONS
# -----------------------------
def lookup_price(area):
    for category, data in WINDOW_PRICE_DB.items():
        if data["min_area"] < area <= data["max_area"]:
            return area * data["price_per_m2"]
    return area * 18000


def categorize_window(area):
    for category, data in WINDOW_PRICE_DB.items():
        if data["min_area"] < area <= data["max_area"]:
            return category
    return "unknown"


def window_type_label(category):
    if category == "small":
        return "Small (≤ 1.5 m²)"
    elif category == "medium":
        return "Medium (1.5 > 3.0 m²)"
    elif category == "large":
        return "Large (≥ 3.0 m²)"
    else:
        return category.capitalize()


def get_material(w):
    material_name = None
    if hasattr(w, "HasAssociations"):
        for rel in w.HasAssociations:
            if hasattr(rel, "RelatingMaterial"):
                material_name = getattr(rel.RelatingMaterial, "Name", None)
    return material_name


def match_size(w, claimed_window, tol=10):
    """
    Check if a single IFC window matches a claimed window, considering units_per_window.
    Assumes horizontal assembly (width multiplied by units_per_window).
    """
    width = getattr(w, "OverallWidth", None)
    height = getattr(w, "OverallHeight", None)
    if width and height:
        # Scale the claimed width by the number of units
        scaled_width = claimed_window["width_mm"] * claimed_window.get("units_per_window", 1)
        return abs(width - scaled_width) <= tol and abs(height - claimed_window["height_mm"]) <= tol
    return False


def match_material(w, claimed_window):
    material_name = get_material(w)
    return material_name == claimed_window["material"]


# Build CLAIMED_TYPE_COUNTS dynamically from CLAIMED_WINDOWS_DB
CLAIMED_TYPE_COUNTS = Counter()
for cw in CLAIMED_WINDOWS_DB:
    area = (cw["width_mm"] / 1000) * (cw["height_mm"] / 1000) * cw.get("units_per_window", 1)
    category = categorize_window(area)
    CLAIMED_TYPE_COUNTS[category] += cw["count"]


# -----------------------------
# MAIN SCRIPT
# -----------------------------

def main():
    model = ifcopenshell.open(r"C:\Users\thoma\Downloads\25-16-D-ARCH.ifc") # PASTE LOCAL FILE NAME
    windows = model.by_type("IfcWindow")
    window_count = len(windows)
    total_area = 0.0
    total_estimated_cost = 0.0
    type_counts = Counter()

    # Prepare matching counters for each claimed window
    match_results = []
    for cw in CLAIMED_WINDOWS_DB:
        match_results.append({
            "window": cw,
            "size_match": 0,
            "material_match": 0,
            "both_match": 0
        })

    # Process each IFC window
    for w in windows:
        height = getattr(w, "OverallHeight", None)
        width = getattr(w, "OverallWidth", None)

        if height and width:
            area = (height / 1000) * (width / 1000)
            total_area += area
            price = lookup_price(area)
            total_estimated_cost += price

            category = categorize_window(area)
            type_counts[category] += 1

            for result in match_results:
                cw = result["window"]
                if match_size(w, cw):
                    result["size_match"] += 1
                if match_material(w, cw):
                    result["material_match"] += 1
                if match_size(w, cw) and match_material(w, cw):
                    result["both_match"] += 1

    # Compute claimed totals (consider units_per_window in area, but count as single window)
    claimed_total_windows = sum(cw["count"] for cw in CLAIMED_WINDOWS_DB)
    claimed_total_area = sum((cw["width_mm"]/1000)*(cw["height_mm"]/1000)*cw.get("units_per_window",1)*cw["count"] for cw in CLAIMED_WINDOWS_DB)
    claimed_total_cost = sum(cw["count"] * cw["price_per_window"] for cw in CLAIMED_WINDOWS_DB)

    # -----------------------------
    # SUMMARY
    # -----------------------------
    print("\n--- SUMMARY ---")
    print(f"IFC total windows: {window_count}")
    print(f"Claimed in report: {claimed_total_windows}")
    print("----------")
    print(f"IFC total cost: {total_estimated_cost:,.0f} DKK")
    print(f"Claimed total cost: {claimed_total_cost:,.0f} DKK")
    print("----------")
    print(f"IFC total window area: {total_area:.2f} m²")
    print(f"Claimed total window area: {claimed_total_area:.2f} m²")
    print("----------\n")

    print("--- WINDOW TYPES IN IFC ---")
    for category in ["small", "medium", "large"]:
        count_ifc = type_counts.get(category, 0)
        print(f"{window_type_label(category):<20}: {count_ifc} windows")

    print("\n--- CLAIMED WINDOW TYPES ---")
    for category in ["small", "medium", "large"]:
        count_claimed = CLAIMED_TYPE_COUNTS.get(category, 0)
        print(f"{window_type_label(category):<20}: {count_claimed} windows")

    print("\n--- CLAIMED WINDOW MATCH ---")
    for result in match_results:
        cw = result["window"]
        print(f"Claimed window: {cw['name']}, {cw['width_mm']} x {cw['height_mm']} mm, Material: {cw['material']}")
        print(f"Matching windows by size: {result['size_match']}")
        print(f"Matching windows by material: {result['material_match']}")
        print(f"Matching windows by size AND material: {result['both_match']}")
        print(f"Claimed in report: {cw['count']}\n")


if __name__ == "__main__":
    main()



--- SUMMARY ---
IFC total windows: 666
Claimed in report: 330
----------
IFC total cost: 9,139,902 DKK
Claimed total cost: 5,941,848 DKK
----------
IFC total window area: 1499.67 m²
Claimed total window area: 1397.23 m²
----------

--- WINDOW TYPES IN IFC ---
Small (≤ 1.5 m²)    : 3 windows
Medium (1.5 > 3.0 m²): 663 windows
Large (≥ 3.0 m²)    : 0 windows

--- CLAIMED WINDOW TYPES ---
Small (≤ 1.5 m²)    : 0 windows
Medium (1.5 > 3.0 m²): 0 windows
Large (≥ 3.0 m²)    : 330 windows

--- CLAIMED WINDOW MATCH ---
Claimed window: Window set: 2 x Udskifte fast vindue af træ, 1.188 x 1.188 mm, 3 lags; 1 x Udskifte vendevindue af træ, 1.188 x 1.188, 3 lags, 1188 x 1188 mm, Material: Wood
Matching windows by size: 0
Matching windows by material: 0
Matching windows by size AND material: 0
Claimed in report: 330

